In [1]:
import os
import glob
import pydicom
import cv2
import numpy as np
from read_roi import read_roi_zip
import zipfile

In [2]:
def get_LUT_value(data, window, level):
    try:
        window = 350
    except TypeError:
        pass
    try:
        level = 50
    except TypeError:
        pass

    return np.piecewise(data,
                        [data <= (level - 0.5 - (window - 1) / 2),
                         data > (level - 0.5 + (window - 1) / 2)],
                        [0, 255, lambda data: ((data - (level - 0.5)) /
                         (window - 1) + 0.5) * (255 - 0)])

def CreateFolder(directory):
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print("Error:Creating dirextory."+directory)

In [3]:
dcm_path = 'D:/appendicitis/1_data_raw/data/Appendicitis/' # dcm 파일 폴더
file_extention = '.png' # 파일 확장자
file_dcm_path = [] # jpg 저장할 환자 폴더
roi_zip_path=[]
dcm_file_name = os.listdir(dcm_path) # 환자 폴더
dcm_file_name.sort()

for i in range(len(dcm_file_name)):
    name = dcm_path + dcm_file_name[i] + '/'
    file_dcm_path.append(name)
    roi_zip = dcm_path + dcm_file_name[i] + '/' + dcm_file_name[i] + '.zip'
    roi_zip_path.append(roi_zip)
# print(len(file_dcm_path), file_dcm_path)
# print(len(roi_zip_path),roi_zip_path)

In [4]:
zip_list_all = [] # roi 이름 저장

for i in range(len(roi_zip_path)):
    zip_file = zipfile.ZipFile(roi_zip_path[i])
    zip_list = zip_file.namelist()
    zip_list.sort()
    zip_list_all.append(zip_list)

In [ ]:
dir_path='D:/appendicitis/1_data_rgb/Appendicitis_rgb_png_256/'
CreateFolder(dir_path)
counts=[]
for i in range(len(file_dcm_path)):
    count=0
    count1=0
    dcm = glob.glob(file_dcm_path[i]+ '/' + '*.dcm')
    for k in range(len(zip_list_all[i])):
        roi_name = zip_list_all[i][k][:4]
        #print(roi_name)
        dcm_name = str(int(roi_name)-1).zfill(4)
        if len(zip_list_all[i])>1:
            if k==0:
                dcm_name_before = str(int(roi_name)-2).zfill(4)
                path_before = file_dcm_path[i] + dcm_file_name[i] +'_'+ dcm_name_before + '.dcm'
            elif k==len(zip_list_all[i])-1:
                dcm_name_after=str(int(roi_name)).zfill(4)
                path_after = file_dcm_path[i] + dcm_file_name[i] +'_'+ dcm_name_after + '.dcm'
        else:
            dcm_name_before = str(int(roi_name)-2).zfill(4)
            path_before = file_dcm_path[i] + dcm_file_name[i] +'_'+ dcm_name_before + '.dcm'
            dcm_name_after=str(int(roi_name)).zfill(4)
            path_after = file_dcm_path[i] + dcm_file_name[i] +'_'+ dcm_name_after + '.dcm'
        #print(dcm_name)
        path = file_dcm_path[i] + dcm_file_name[i] +'_'+ dcm_name + '.dcm'
        #print(path)
        try:
            ds = pydicom.read_file(path)
        except:
            path = file_dcm_path[i] + dcm_file_name[i] + dcm_name + '.dcm'
            print(path)
            ds=pydicom.read_file(path)
        ArrayDicom = ds.pixel_array
        ArrayDicom = ds.RescaleSlope*ArrayDicom+ds.RescaleIntercept
        ArrayDicom = get_LUT_value(ArrayDicom, ds.WindowWidth, ds.WindowCenter) #설정은 함수에서 조절하기
        file_name = dcm_file_name[i] +'_'+ dcm_name
        CreateFolder(dir_path+dcm_file_name[i]+'/')
        cv2.imwrite(dir_path+dcm_file_name[i]+'/'+file_name+'.png', ArrayDicom)
        count+=1
        count1+=1
        cv2.imwrite(file_name+'.png', ArrayDicom)
    #print(path_before, path_after)
    bfs=[path_before,path_after]
    name=[dcm_name_before,dcm_name_after]
    m=0
    for bf in bfs:
        path = bf
        #print(path)
        try:
            ds = pydicom.read_file(path)
        except:
            path = file_dcm_path[i] + dcm_file_name[i] + name[m] + '.dcm'
            print(path)
            ds=pydicom.read_file(path)
        ArrayDicom = ds.pixel_array
        ArrayDicom = ds.RescaleSlope*ArrayDicom+ds.RescaleIntercept
        ArrayDicom = get_LUT_value(ArrayDicom, ds.WindowWidth, ds.WindowCenter) #설정은 함수에서 조절하기
        file_name = dcm_file_name[i] +'_'+ name[m]
        CreateFolder(dir_path+dcm_file_name[i]+'/')
        cv2.imwrite(dir_path+dcm_file_name[i]+'/'+file_name+'.png', ArrayDicom)
        m+=1
        count+=1
    counts.append(count)
    #print(count, count1, count-count1)


# 영역 crop 

# DICOM 파일 경로와 ROI zip 경로 저장 

In [8]:
dcm_path = 'D:/appendicitis/1_data_rgb/Appendicitis_rgb_png_256/' # png 파일 폴더
file_extention = '.png' # 파일 확장자
file_dcm_path = [] # jpg 저장할 환자 폴더
roi_zip_path=[]
dcm_file_name = os.listdir(dcm_path) # 환자 폴더
dcm_file_name.sort()

for i in range(len(dcm_file_name)):
    name = dcm_path + dcm_file_name[i] + '/'
    file_dcm_path.append(name)
    roi_zip = dcm_path + dcm_file_name[i] + '/' + dcm_file_name[i] + '.zip'
    roi_zip_path.append(roi_zip)
# print(len(file_dcm_path), file_dcm_path)
# print(len(roi_zip_path),roi_zip_path)

# ROI 정보 저장

In [9]:
zip_list_all = [] # roi 이름 저장
roi_zip_path=glob.glob('D:/appendicitis/1_data_raw/data/Appendicitis_roi/'+'*.zip')
for i in range(len(roi_zip_path)):
    zip_file = zipfile.ZipFile(roi_zip_path[i])
    zip_list = zip_file.namelist()
    zip_list.sort()
    zip_list_all.append(zip_list)

In [10]:
len(zip_list_all)

246

# Crop

In [ ]:
import random
mask_path = 'D:/appendicitis/1_data_rgb/Appendicitis_rgb_crop_256/'
dir_path='D:/appendicitis/1_data_rgb/Appendicitis_rgb_png_256/'
CreateFolder(mask_path)
a=[0,-1]
for i in range(len(zip_list_all)):
#for i in range(1):
    for k in range(len(zip_list_all[i])):
        png_path=dir_path+dcm_file_name[i]+'/'
        file = glob.glob(png_path+'*.png')
        roi = read_roi_zip(roi_zip_path[i])
        name = zip_list_all[i][k][:14]
        img = cv2.imread(file[k+1])
        left_top = roi[name]['left'], roi[name]['top']
        right_bottom = left_top[0]+roi[name]['width'], left_top[1]+roi[name]['height']
#         print(left_top,right_bottom)
        crop=img.copy()
        crop_img=crop[left_top[1]:right_bottom[1],left_top[0]:right_bottom[0]]
        dcm_name = str(int(zip_list_all[i][k][:4])-1).zfill(4)
        file_name = dcm_file_name[i] +'_'+ dcm_name
        print(file_name)
        CreateFolder(mask_path+'/'+dcm_file_name[i]+'/')
        cv2.imwrite(mask_path+'/'+dcm_file_name[i]+'/'+file_name+'.png', crop_img)
    
    for m in a:
        #print(m)
        roi = read_roi_zip(roi_zip_path[i])
        name = zip_list_all[i][m][:14]
        img = cv2.imread(file[m])
        left_top = roi[name]['left'], roi[name]['top']
        right_bottom = left_top[0]+roi[name]['width'], left_top[1]+roi[name]['height']
    #         print(left_top,right_bottom)
        crop=img.copy()
        crop_img=crop[left_top[1]:right_bottom[1],left_top[0]:right_bottom[0]]
        if m==a[0]:
            dcm_name = str(int(zip_list_all[i][m][:4])-2).zfill(4)
            file_name = dcm_file_name[i] +'_'+ dcm_name
            print(file_name)
        elif m==a[-1]:
            dcm_name = str(int(zip_list_all[i][m][:4])).zfill(4)
            file_name = dcm_file_name[i] +'_'+ dcm_name
            print(file_name)
        CreateFolder(mask_path+'/'+dcm_file_name[i]+'/')
        cv2.imwrite(mask_path+'/'+dcm_file_name[i]+'/'+file_name+'.png', crop_img)